### *Maciej Pawlikowski*
# AI4Games: projekt

Byłem członkiem grupy zajmującej się tworzeniem generatora map losowych do Heroes of Might and Magic III. Moja rola w projekcie polegała na napisaniu uproszczonej wersji silnika walki, a następnie wyznaczeniu współczynników odzwierciedlających wzajemną siłę potworków występujących w grze. Współczynniki te (prawdopodobnie) będą pomocne w 
podejmowaniu decyzji o rodzaju i liczbie stworów stawianych na planszy w procesie wypełniania mapy obiektami.

### Wartości jednostek

Sama gra posiada wewnętrzne wartości, które wykorzystuje do szacowania siły armii. Szczególnie interesującym atrybutem jest "AI Value", które na pierwszy rzut oka wygląda dosyć sensownie. Niestety każdy potwór ma tylko jedno AI Value. Z tego powodu nie można na podstawie tej wartości określić jak (w stosunku do innych monstrów) poradzi sobie w walce przeciwko pewnemu ograniczonemu podzbiorowi wszystkich jednostek, na przykład przeciwko jednostkom neutralnym, albo jednostkom konkretnego poziomu.

Wydaje się, że dobre oszacowanie wzajemnej siły kreatur można by uzyskać mając osobny współczynnik dla każdej *pary* jednostek. Pomogłoby to również wyznaczać względną jakość jednostek przy ograniczeniu do określonej grupy przeciwników. W tym projekcie przedstawiam propozycję takiego właśnie wartościowania.

### Metoda

Szacowanie przeprowadzone zostało przy założeniu, że siła jednostek jest addytywna, czyli wartość armii to suma wartości pojedynczych jednostek, którę tę armię tworzą. Wydaje się to dosyć sensowne, chociaż niektóre kombinacje jednostek są wyjątkowo efektywne (na przykład Strzelcy w połączeniu z wolnymi, wytrzymałymi jednostami), więc robimy tutaj pewne uproszczenie. Założenie poprowadziło do pomysłu przeprowadzenia symulacji walk jednostek, w których po każdej stronie walczy tylko jeden oddział (oczywiście złożony z wielu takich samych potwórów). Starałem się dla każdej pary bestii znaleźć takie liczby jednostek w oddziałach, że obie strony mają mniej więcej równe szanse. Algorytm przedstawia się tak:
   - $A$, $B \leftarrow$ rodzaje walczących jednostek
   - $S_A$, $S_B \leftarrow$ oddziały z pewnymi początkowymi liczbami stworków
   - symuluj walkę $S_A$ z $S_B$, wyznacz $S_w$, $S_l$ - wygrywający i przegrywający oddział ($\lbrace S_A, S_B \rbrace = \lbrace S_w, S_l \rbrace$)
   - $low \leftarrow count(S_l)$
   - dopóki $S_l$ przegrywa z $S_w$:
       - $low \leftarrow count(S_l)$
       - zwiększ $count(S_l)$ (w kodzie zwięszam o arbitralne $10 \%$ wartości początkowej)
   - szukaj w $(low, count(S_l))$ takiej liczby $k$ potworów w $S_l$, dla której wynik walki $S_l$ z $S_w$ jest wyrównany
   - ustaw $count(S_l) = k$
   - zwróć $\frac{count(S_B)}{count(S_A)}$ (cały czas $\lbrace S_A, S_B \rbrace = \lbrace S_w, S_l \rbrace$)
   
W każdym miejscu, w którym pojawia się symulowana walka między oddziałami, przeprowadzam $500$ walk (liczba dobrana na oko) i zliczam wygrane każdego z oddziałów. Oddział przegrywa walkę, gdy wygrał $<250$ (mniej niż połowę) pojedynczych walk. Wynik walki uznaję za "wyrównany", gdy różnica między liczbami wygranych obu oddziałów jest $<25$ (czyli jego wynik to połowa $\pm 5 \%$).

Trzeba jeszcze jakoś wyznaczyć początkowe wartości oddziałów. Skorzystałem w tym celu z AI Value. W walce $A$ z $B$ ustawiam $count(S_A) = AI\_Value(B)$ oraz $count(S_B) = AI\_Value(A)$.

Nie przeprowadzałem symulacji walk takich samych jednostek, wtedy oczywiście współczynnik wynosi $1$.

### Walka

*Dalej przez *strzelca* rozumiem jednostkę posiadającą atak dystansowy, a przez *piechura* taką, która go nie posiada. Tak, kawalerzysta również jest *piechurem* na potrzeby tego tekstu.*

Silnik walki został znacznie uproszczony, ale w znacznej większości przypadków powinien działać identycznie (wyjątki opisane w sekcji **Zdolności specjalne**). Wyróżniam trzy podstawowe przypadki:

    1) piechur - piechur
    2) strzelec - strzelec
    3) piechur - strzelec
    
**1) Piechur - piechur**

Prosty przypadek, jednostki atakują się wzajemnie dopóki któraś nie padnie. Pierwszy zawsze atakuje oddział o większej szybkości, remisy rozstrzygane są losowo.

**2) Strzelec - strzelec**

Oddziały strzelają do siebie nawzajem dopóki któremuś nie skończą się strzały, później przypadek (3). Kolejność ataków jak w (1).

**3) Piechur - strzelec**

Skomplikowane. W skrócie: biorę pod uwagę dokładnie ilu strzałów z pełną skutecznością piechur może uniknąć, jeśli jest sprytny. 

Wersja dłuższa:

    dist to odległość-1, czyli liczba pól, jakie piechur musi przejść
    ps = szybkość piechura

    jeśli piechur jest wolniejszy niż strzelec (*):
      strzelec strzela (dist / ps + (dist % ps > 0)) razy

      Piechur może uniknąć najwyżej jednego "pełnego" strzału. 
      Unika go wtedy, kiedy w pierwszym ruchu nie wejdzie w zasięg strzelca. 
      Optymalny pierwszy ruch to
          (dist - dist % ps) lub ps gdy to pierwsze jest zerem.
          
    jeśli piechur jest szybszy (**):
      strzelec strzela (dist / ps - (dist % ps == 0)) razy

      Piechur unika dodatkowo jednego pełnego strzału (jednego jak w (*), jednego czekając w walce).
      Czyli unika jednego lub dwóch.
      
    wpp:
      Piechur nie może uniknąć strzału czekając, bo żadnej ze stron nie opłaca się czekać (czekanie zmienia 
      kolejność ruchów jednostek w następnej rundzie).

      Strzelec strzela tyle razy co w (**) (jeśli zaczyna) i tyle co w (*) (jeśli nie).


    Potem przypadek (2), piechur zaczyna.
    
W pseudokodzie nie widać wyraźnego podziału na rundy, bo właśnie na tym polegało uproszczenie systemu walki. Nie ma rund, tylko opisanie tych trzech przypadków w sposób równoważny i krótszy (na przykład pomijam ruch jednostek). To spowodowało pewne trudności w implementacji niektórych zdolności specjalnych działających przez pewną liczbę rund. Nie były to jednak wielkie problemy.

### Combat rules

I took into account every basic combat mechanic and creature characteristic, like attack, defense, number of shots, retaliation, creature size, etc. In addition to that, I implemented most special abilities:

  - no enemy retaliation
  - no meele penalty
  - double attack or shot
  - rebirth (Phoenix)
  - enemy defense reduction (Behemoth, Ancient Behemoth)
  - death blow (Dread Knight)
  - life drain (Vampire Lord)
  - death stare (Mighty Gorgon)
  - hate (e.g. Angel - Devil) and double damage (opposing Elementals)
  - fear (Azure Dragon)
  - regeneration (Wight, Wraith, Troll)
  - fire shield (Efreet Sultan)
  - acid breath (Rust Dragon)
  - lighning strike (Thunderbird)
  - aging (Ghost Dragon)
  - poison (Wyvern Monarch)
  - curse (Mummy, Black Knight, Dread Knight) (small discrepancy in duration - sometimes ends half a round too early)
  - weakness (Dragon Fly) (same as above)
  - disease (Zombie) (same as above)
  - blind (Unicorn, War Unicorn)
  - paralyzing venom (Scorpicore)
  - petrification (all Basilisks and Medusas)
  
In all cases (I hope) I consider the following creatures' immunities:

  - golems - less magic damage
  - Firebird, Phoenix, Efreet, Efreet Sultan - immune to fire spells
  - all undeads - resistances of undeads
  - all elementals - resistances of elementals
  - gargoyles, golems - resistances of non-living creatures
  - Green Dragon, Red Dragon, Azure Dragon - immune to spells 1-3
  - Gold Dragon - immune to spells 1-4
  - Black Dragon, Magic Elemental - immune to all spells
  - Dwarf, Battle Dwarf, Crystal Dragon - magic resistance (%)
  - Giant, Titan - immune to mind spells
  - Troglodytes, Infernal Troglodytes - immune to blind and petrification
  
Not implemented:

  - Faerie Dragon's spells - a lot of work, not much payoff (very rare creature)
  - jousting (Cavalier, Champion)
  - strike and return (Harpy Hag)
  - abilities that are useless in a duel
  - everything I forgot

### Various stuff

Każda wartość zamieszczona w pliku *scores.csv* wyznacza iloraz liczebności oddziałów, przy którym wynik walki jest wyrównany. Jest tylko jeden problem: te wartości są stabilne dla dużych oddziałów. W przypadku walk kameralnych sprawa się komplikuje. Na przykład gdy pikinierzy zadadzą 700 obrażeń, to zabiją 2 Archanioły. Gdy zadzadzą 10 razy więcej, zabiją 28, czyli aż 14 razy więcej Archaniołów. Prowadzi to do tego, że 8 Archaniołów klepie bez problemu 510 Pikinierów, ale 80 Archaniołów i 5100 Pikinierów mają w miarę równe szanse. Wynika to z tego, że słabe uderzenia nie mają żadnego wpływu na zdolności bojowe jednostek (oddział atakuje słabiej dopiero, gdy przynajmniej jedna jednostka zginie). W związku z tym wyznaczone wartości będą faworyzowały jednostki wysokiego poziomu.

*-*

Obliczenie wszystkich wartości trwało około 5 godzin na nowoczesnym i7, bez żadnych optymalizacji, w Pythonie.

*-*

Mając dany pozdbiór stworków $A$ można, ze stratą informacji, wyznaczyć dla każdego potwora w grze jedną liczbę reprezentującą jego siłę w walce przeciwko $A$. Wystarczy wziąć sumę jego wspóczynników po wszystkich potworach w $A$. Po odpowiednim przeskalowaniu, tak obliczone wartości, dla $A$ będącego zbiorem wszystkich występujących w grze potworów, można bezpośrednio porównywać z AI Value. Mogą też one posłużyć jako parametry jakiegoś rozkładu prawdopodobieństwa, jeśli będziemy chcieli sensownie wylosować potworka, którego postawić na podwórku gracza.

*-*

Oprócz symulatora walk zrobiłem też prowizoryczny klikalny interfejs, w którym można przeglądać parametry potworków i wyniki walk oraz obliczać wartości opisane w poprzednim punkcie. Mam zamiar go jeszcze porządnie rozwinąć, ale na razie nie starczyło mi czasu.

*-*

Krótki Opis Ważnych Plików:

    combat_sim.ipynb - notebook do eksperymentów
    combat.py - kod walk 
    unit.py - reprezentacja jednostki i oddziału
    scores.csv - wyniki
    CRTRAITS.TXT - parametry potworków, zgodne z najnowszą angielską wersją
    
*-*

Przydatne linki:

http://h3.heroes.net.pl/uploaded/download/Heroes33patch.ZIP - nieoficjalny patch do polskiej wersji zawierający aktualne parametry stworów

http://heroes.thelazy.net/ - obszerne źródło informacji o H3

http://heroescommunity.com/viewthread.php3?TID=19321&pagenumber=2 - opis CRTRAITS.TXT

http://heroescommunity.com/viewthread.php3?TID=12210&pagenumber=2 - dyskusja o tym, jak zabijają krowy